In [1]:
# imports
from shutil import copyfileobj
from urllib import request
import pandas as pd
import uuid
import xlwings as xw
from xlwings import load
import re
import datetime
import pytz as tz
from bs4 import BeautifulSoup
import requests

In [2]:
#%env XLWINGS_LICENSE_KEY=noncommercial
#!xlwings license update -k noncommercial

In [3]:
URL_DOMAIN = "https://www.vendeeglobe.org"
URL_CLASSEMENT = "https://www.vendeeglobe.org/fr/classement"
URL_DOWNLOAD = "/download-race-data/"
#URL_DOWNLOAD = "/download-race-data/vendeeglobe_20210305_080000.xlsx"
URL_VOILIERS = "https://www.vendeeglobe.org/fr/glossaire"

EXCEL_FILEPATH = './data/excel/'
CSV_FILEPATH = './data/csv/'

filename_arrivee = 'arrivee'
filename_exemple = 'exemple'

In [4]:
def read_xlsx_to_dataframe(filepath):
    excel_app = xw.App(visible=False)
    #excel_book = excel_app.books.open(filepath)
    excel_book = xw.Book(filepath, mode='r')
    sheet1 = excel_book.sheets[0]
    #cells = sheet1.used_range
    #df = cells.value
    df = sheet1.cells.options('df').value
    excel_book.close()
    excel_app.quit()
    df.columns = range(df.columns.size)
    df.reset_index(inplace=True, drop=True)
    #return pd.DataFrame(df)
    return df

def clean_all_null(df):
    
    cleaned_df = df.copy()
    
    none_rows = []
    for i in range(df.shape[0]):
        if pd.isnull(cleaned_df.iloc[i,:]).all():
            none_rows.append(i)
    if len(none_rows) > 0:
        cleaned_df.drop(none_rows,axis=0, inplace=True)
    
    none_columns = []
    for j in range(df.shape[1]):
        if pd.isnull(cleaned_df.iloc[:,j]).all():
            none_columns.append(j)
    if len(none_columns) > 0:
        cleaned_df.drop(none_columns,axis=1, inplace=True)
    
    cleaned_df.reset_index(inplace=True, drop=True)
    cleaned_df.columns = range(cleaned_df.columns.size)
    
    return cleaned_df

def separate_metadata(df):
    cleaned_df = df.copy()
    
    metadata = {}
    
    # get classement_date information
    metadata["classement_date"] = cleaned_df.iat[0,0]
    cleaned_df.iat[0,0] = None
    
    # get distance definition
    metadata["distance_definition"] = cleaned_df.iat[-1,0]
    cleaned_df.iat[-1,0] = None
    
    # get distance definition
    metadata["velocity_definition"] = cleaned_df.iat[-2,0]
    cleaned_df.iat[-2,0] = None
    
    # get calculation services
    metadata["calculation_services"] = cleaned_df.iat[-3,0]
    cleaned_df.iat[-3,0] = None
    
    return (cleaned_df, metadata)

In [5]:
#data = read_xlsx_to_dataframe(EXCEL_FILEPATH + filename_exemple + '.xlsx')

In [6]:
#cleaned_data = clean_all_null(data)
#cleaned_data, metadata = separate_metadata(cleaned_data)
#cleaned_data = clean_all_null(cleaned_data)

In [7]:
data_definition_arrivee = {
    0:{"header_alias":"Rang", "raw_headers":[(0,0)], "data_col":0, "data_start_row":2, "force_alias":False},
    1:{"header_alias":"Nat/Voile", "raw_headers":[(0,1)], "data_col":1, "data_start_row":2, "force_alias":False},
    2:{"header_alias":"Skipper/Bateau", "raw_headers":[(0,2)], "data_col":2, "data_start_row":2, "force_alias":False},
    3:{"header_alias":"Date d'Arrivée", "raw_headers":[(0,3)], "data_col":4, "data_start_row":2, "force_alias":False},
    4:{"header_alias":"Temps de course", "raw_headers":[(0,5)], "data_col":5, "data_start_row":2, "force_alias":False},
    5:{"header_alias":"Ecarts au premier", "raw_headers":[(0,6),(1,6)], "data_col":7, "data_start_row":2, "force_alias":False},
    6:{"header_alias":"Ecarts au précédent", "raw_headers":[(0,6),(1,8)], "data_col":9, "data_start_row":2, "force_alias":False},
    7:{"header_alias":"Sur l'ortho - Vitesse", "raw_headers":[(0,10),(1,10)], "data_col":10, "data_start_row":2, "force_alias":False},
    8:{"header_alias":"Sur l'ortho - Distance", "raw_headers":[(0,10),(1,11)], "data_col":11, "data_start_row":2, "force_alias":False},
    9:{"header_alias":"pourcentage", "raw_headers":[(0,12)], "data_col":12, "data_start_row":2, "force_alias":True},
    10:{"header_alias":"Sur le fond - Vitesse", "raw_headers":[(0,13),(1,13)], "data_col":13, "data_start_row":2, "force_alias":False},
    11:{"header_alias":"Sur le fond - Distance", "raw_headers":[(0,13),(1,14)], "data_col":14, "data_start_row":2, "force_alias":False}
}

data_definition = {
    0:{"header_alias":"rang", "raw_headers":[(0,0)], "data_col":0, "data_start_row":2, "force_alias":False},
    1:{"header_alias":"nat_voile", "raw_headers":[(0,1)], "data_col":1, "data_start_row":2, "force_alias":False},
    2:{"header_alias":"skipper_bateau", "raw_headers":[(0,2)], "data_col":2, "data_start_row":2, "force_alias":False},
    3:{"header_alias":"heure_fr", "raw_headers":[(1,3)], "data_col":3, "data_start_row":2, "force_alias":False},
    4:{"header_alias":"latitude", "raw_headers":[(1,4)], "data_col":4, "data_start_row":2, "force_alias":False},
    5:{"header_alias":"longitude", "raw_headers":[(1,5)], "data_col":5, "data_start_row":2, "force_alias":False},
    6:{"header_alias":"depuis_30_min_cap", "raw_headers":[(0,6),(1,6)], "data_col":6, "data_start_row":2, "force_alias":False},
    7:{"header_alias":"depuis_30_min_vitesse", "raw_headers":[(0,6),(1,7)], "data_col":7, "data_start_row":2, "force_alias":False},
    8:{"header_alias":"depuis_30_min_vmg", "raw_headers":[(0,6),(1,8)], "data_col":8, "data_start_row":2, "force_alias":False},
    9:{"header_alias":"depuis_30_min_distance", "raw_headers":[(0,6),(1,9)], "data_col":9, "data_start_row":2, "force_alias":False},
    10:{"header_alias":"depuis_dernier_classement_cap", "raw_headers":[(0,10),(1,10)], "data_col":10, "data_start_row":2, "force_alias":False},
    11:{"header_alias":"depuis_dernier_classement_vitesse", "raw_headers":[(0,10),(1,11)], "data_col":11, "data_start_row":2, "force_alias":False},
    12:{"header_alias":"depuis_dernier_classement_vmg", "raw_headers":[(0,10),(1,12)], "data_col":12, "data_start_row":2, "force_alias":False},
    13:{"header_alias":"depuis_dernier_classement_distance", "raw_headers":[(0,10),(1,13)], "data_col":13, "data_start_row":2, "force_alias":False},
    14:{"header_alias":"depuis_24_heure_cap", "raw_headers":[(0,14),(1,14)], "data_col":14, "data_start_row":2, "force_alias":False},
    15:{"header_alias":"depuis_24_heure_vitesse", "raw_headers":[(0,14),(1,15)], "data_col":15, "data_start_row":2, "force_alias":False},
    16:{"header_alias":"depuis_24_heure_vmg", "raw_headers":[(0,14),(1,16)], "data_col":16, "data_start_row":2, "force_alias":False},
    17:{"header_alias":"depuis_24_heure_distance", "raw_headers":[(0,14),(1,17)], "data_col":17, "data_start_row":2, "force_alias":False},
    18:{"header_alias":"dtf", "raw_headers":[(0,18)], "data_col":18, "data_start_row":2, "force_alias":False},
    19:{"header_alias":"dtl", "raw_headers":[(0,19)], "data_col":19, "data_start_row":2, "force_alias":False}
}

In [8]:
def format_header(df, data_definition, use_alias):
    
    formatted_df = pd.DataFrame()
    
    for key in data_definition.keys():
        
        data_r = data_definition[key]["data_start_row"]
        data_c = data_definition[key]["data_col"]
        force_alias = data_definition[key]["force_alias"]
        
        if use_alias or force_alias:
            alias = data_definition[key]["header_alias"]
            formatted_df[alias] = df[data_c].iloc[data_r:]
        else:
            raw_header = ""
            for indices in data_definition[key]["raw_headers"]:
                raw_header = raw_header + df.iloc[indices[0], indices[1]]
                
            formatted_df[raw_header] = df[data_c].iloc[data_r:]
            
    formatted_df.reset_index(inplace=True, drop=True)
    
    return formatted_df

#formatted_header_data = format_header(cleaned_data, data_definition, True)


In [9]:
def format_rang(df, rang_col_name):
    if rang_col_name in df.columns:
        df[rang_col_name] = df[rang_col_name].str.replace('\nARV','')
        df[rang_col_name] = df[rang_col_name].str.replace('RET',str(-1))
        
#format_rang(formatted_header_data, 'rang')

def format_nat_voile(df, nat_voile_col_name, nat_col_name, voile_col_name):
    if nat_voile_col_name in df.columns:
        df[nat_voile_col_name] = df[nat_voile_col_name].str.replace('\n',' ')
        df[nat_voile_col_name] = df[nat_voile_col_name].str.upper()
        df[nat_col_name] = df[nat_voile_col_name].str.extract(r'([A-Z]+)')
        df[voile_col_name] = df[nat_voile_col_name].str.extract(r'([0-9]+)')
        df.drop(nat_voile_col_name, axis=1, inplace=True)

#format_nat_voile(formatted_header_data, 'nat_voile', 'nationalite', 'voile')

def format_skipper_bateau(df, skipper_bateau_col_name, skipper_col_name, bateau_col_name):
    if skipper_bateau_col_name in df.columns:
        df[skipper_col_name] = df[skipper_bateau_col_name].str.split('\n').apply(lambda x: x[0])
        df[bateau_col_name] = df[skipper_bateau_col_name].str.split('\n').apply(lambda x: x[1])
        df.drop(skipper_bateau_col_name, axis=1, inplace=True)
    
#format_skipper_bateau(formatted_header_data, 'skipper_bateau', 'skipper', 'bateau')


In [10]:
def format_date_classement(metadata):
    classement_date = metadata['classement_date']
    
    months_dict = {
        'janvier':'01',
        'février':'02',
        'mars':'03',
        'avril':'04',
        'mai':'05',
        'juin':'06',
        'juillet':'07',
        'août':'08',
        'septembre':'09',
        'octobre':'10',
        'novembre':'11',
        'décembre':'12'
    }
    
    day_pattern = re.compile('lundi|mardi|mercredi|jeudi|vendredi|samedi|dimanche')
    month_pattern = re.compile('janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre')
    
    day = re.search(day_pattern, classement_date).group()
    date = re.search('[0-9]+', re.search(' [0-9][0-9]? ', classement_date).group()).group()
    month = re.search(month_pattern, classement_date).group()
    year = re.search('[0-9]{4}', classement_date).group()
    dd_mm_yyyy = str(date) + '/' + str(months_dict[month]) + '/' + str(year)

    time_place = re.search('[0-9]{2}h[0-9]{2} [A-Z]+', classement_date).group()
    time = re.search('[0-9]{2}h[0-9]{2}', time_place).group().replace('h',':')
    place = re.search('[A-Z]+', time_place).group()

    metadata['day'] = day
    metadata['date'] = date
    metadata['month'] = month
    metadata['year'] = year
    metadata['dd_mm_yyyy'] = dd_mm_yyyy
    metadata['time'] = time
    metadata['place'] = place

#format_date_classement(metadata)


In [11]:
def format_heure_fr(df, date, heure_fr_col_name):
    #fr_tz = timezone('Europe/Paris')
    #datetime.tzinfo('Europe/Paris')
    if str(type(df['heure_fr'][0])) != "<class 'pandas._libs.tslibs.timestamps.Timestamp'>":
        df[heure_fr_col_name] = df[heure_fr_col_name].replace('[0-9]+min','',regex=True)
        df[heure_fr_col_name] = df[heure_fr_col_name].str.replace('-','')
        df[heure_fr_col_name] = df[heure_fr_col_name].str.replace('\r','')
        df[heure_fr_col_name] = df[heure_fr_col_name].str.replace('\n','')
        df[heure_fr_col_name] = pd.to_datetime(date +' '+ df[heure_fr_col_name])
        
#format_heure_fr(formatted_header_data, metadata['dd_mm_yyyy'], 'heure_fr')

def format_cap(df, cap_column):
    if cap_column in df.columns:
        #df[cap_column] = df[cap_column].str.extract(r'([0-9]+)')
        df[cap_column] = df[cap_column].str.replace('°','')
        
#format_cap(formatted_header_data, 'depuis_30_min_cap')
#format_cap(formatted_header_data, 'depuis_dernier_classement_cap')
#format_cap(formatted_header_data, 'depuis_24_heure_cap')

def format_vitesse(df, vitesse_column):
    if vitesse_column in df.columns:
        #df[vitesse_column] = df[vitesse_column].str.extract(r'([0-9\.]+)')
        df[vitesse_column] = df[vitesse_column].str.replace(' ','')
        df[vitesse_column] = df[vitesse_column].str.replace('kts','')
        
#format_vitesse(formatted_header_data, 'depuis_30_min_vitesse')
#format_vitesse(formatted_header_data, 'depuis_dernier_classement_vitesse')
#format_vitesse(formatted_header_data, 'depuis_24_heure_vitesse')
#format_vitesse(formatted_header_data, 'depuis_30_min_vmg')
#format_vitesse(formatted_header_data, 'depuis_dernier_classement_vmg')
#format_vitesse(formatted_header_data, 'depuis_24_heure_vmg')

def format_distance(df, distance_column):
    if distance_column in df.columns:
        #df[distance_column] = df[distance_column].str.extract(r'([0-9\.]+)')
        df[distance_column] = df[distance_column].str.replace(' ','')
        df[distance_column] = df[distance_column].str.replace('nm','')
        
#format_distance(formatted_header_data, 'depuis_30_min_distance')
#format_distance(formatted_header_data, 'depuis_dernier_classement_distance')
#format_distance(formatted_header_data, 'depuis_24_heure_distance')
#format_distance(formatted_header_data, 'dtf')
#format_distance(formatted_header_data, 'dtl')


In [12]:
def save_dataframe_to_csv(df, filepath):
    df.to_csv(filepath, index=False)
    
#save_dataframe_to_csv(formatted_header_data, CSV_FILEPATH+filename_exemple+'.csv')


In [13]:
def read_csv_to_dataframe(filepath):
    return pd.read_csv(filepath)

#read_csv_to_dataframe(CSV_FILEPATH+filename_exemple+'.csv')


In [14]:
# download excel file from url
#with request.urlopen(URL_DOMAIN + URL_DOWNLOAD) as response, open('./data/excel/exemple.xlsx', 'wb') as out_file:
#    copyfileobj(response, out_file)
    

In [15]:
def get_excel_filenames_from_website(url):
    
    excel_file_names = []
    r = requests.get(url)
    
    if r.status_code != 200:
        raise Exception('Error: Status Code:' + r.status_code)
    
    soup = BeautifulSoup(r.content)
    option_tag_array = soup.find_all('option')
    
    if option_tag_array[0].text != 'Précédents classements':
        raise Exception('Webpage content changed, Précédents classements expected, got ' + option_tag_array[0].text + ' instead')
    
    for element in option_tag_array[1:]:
        excel_file_names.append('vendeeglobe_' + element['value'] + '.xlsx')
        
    return excel_file_names


In [16]:
def download_classement_file(filename):
    
    url = URL_DOMAIN + URL_DOWNLOAD + filename
    download_file(url, EXCEL_FILEPATH + filename)
    
def download_file(url, filename):
    with request.urlopen(url) as response, open(filename, 'wb') as out_file:
        copyfileobj(response, out_file)
    
#for file in files_to_download:
#    download_classement_file(file)

    

In [17]:
def download_clean_save_csv_classement_file(filename):
    print(filename)
    # download xlsx
    download_classement_file(filename)
    # read xlsx
    data = read_xlsx_to_dataframe(EXCEL_FILEPATH + filename)
    # clean xlsx rows and columns
    cleaned_data = clean_all_null(data)
    cleaned_data, metadata = separate_metadata(cleaned_data)
    cleaned_data = clean_all_null(cleaned_data)
    
    # detect fichier arrivee
    if cleaned_data.iloc[0, 3] is not None and 'arrivée' in cleaned_data.iloc[0, 3]:
        print("fichier date d'arrivée: " + filename)
        return 
    
    # format header
    formatted_header_data = format_header(cleaned_data, data_definition, True)
    
    # format columns
    format_rang(formatted_header_data, 'rang')
    format_nat_voile(formatted_header_data, 'nat_voile', 'nationalite', 'voile')
    format_skipper_bateau(formatted_header_data, 'skipper_bateau', 'skipper', 'bateau')
    format_date_classement(metadata)
    format_heure_fr(formatted_header_data, metadata['dd_mm_yyyy'], 'heure_fr')
    format_cap(formatted_header_data, 'depuis_30_min_cap')
    format_cap(formatted_header_data, 'depuis_dernier_classement_cap')
    format_cap(formatted_header_data, 'depuis_24_heure_cap')
    format_vitesse(formatted_header_data, 'depuis_30_min_vitesse')
    format_vitesse(formatted_header_data, 'depuis_dernier_classement_vitesse')
    format_vitesse(formatted_header_data, 'depuis_24_heure_vitesse')
    format_vitesse(formatted_header_data, 'depuis_30_min_vmg')
    format_vitesse(formatted_header_data, 'depuis_dernier_classement_vmg')
    format_vitesse(formatted_header_data, 'depuis_24_heure_vmg')
    format_distance(formatted_header_data, 'depuis_30_min_distance')
    format_distance(formatted_header_data, 'depuis_dernier_classement_distance')
    format_distance(formatted_header_data, 'depuis_24_heure_distance')
    format_distance(formatted_header_data, 'dtf')
    format_distance(formatted_header_data, 'dtl')
    # save to csv
    save_dataframe_to_csv(formatted_header_data, CSV_FILEPATH + filename.replace('.xlsx','.csv'))


In [20]:
def download_files_convert_csv(start_filename=None, stop_filename=None):
    
    files_to_download = get_excel_filenames_from_website(URL_CLASSEMENT)
    
    if start_filename is not None or stop_filename is not None:
        index_start_filename = files_to_download.index(start_filename)
        index_stop_filename = files_to_download.index(stop_filename) + 1
        files_to_download = files_to_download[index_start_filename: index_stop_filename]
        
    for file in files_to_download:
        download_clean_save_csv_classement_file(file)
        
download_files_convert_csv('vendeeglobe_20210103_080000.xlsx', 'vendeeglobe_20201108_120200.xlsx')

vendeeglobe_20210103_080000.xlsx


/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname FR identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


vendeeglobe_20210103_040000.xlsx
vendeeglobe_20210102_210000.xlsx
vendeeglobe_20210102_170000.xlsx
vendeeglobe_20210102_140000.xlsx
vendeeglobe_20210102_110000.xlsx
vendeeglobe_20210102_080000.xlsx
vendeeglobe_20210102_040000.xlsx
vendeeglobe_20210101_210000.xlsx
vendeeglobe_20210101_170000.xlsx
vendeeglobe_20210101_140000.xlsx
vendeeglobe_20210101_110000.xlsx
vendeeglobe_20210101_080000.xlsx
vendeeglobe_20210101_040000.xlsx
vendeeglobe_20201231_210000.xlsx
vendeeglobe_20201231_170000.xlsx
vendeeglobe_20201231_140000.xlsx
vendeeglobe_20201231_110000.xlsx
vendeeglobe_20201231_080000.xlsx
vendeeglobe_20201231_040000.xlsx
vendeeglobe_20201230_210000.xlsx
vendeeglobe_20201230_170000.xlsx
vendeeglobe_20201230_140000.xlsx
vendeeglobe_20201230_110000.xlsx
vendeeglobe_20201230_110000.xlsx
vendeeglobe_20201230_080000.xlsx
vendeeglobe_20201230_040000.xlsx
vendeeglobe_20201229_210000.xlsx
vendeeglobe_20201229_170000.xlsx
vendeeglobe_20201229_140000.xlsx
vendeeglobe_20201229_110000.xlsx
vendeeglob

vendeeglobe_20201122_210000.xlsx
vendeeglobe_20201122_170000.xlsx
vendeeglobe_20201122_140000.xlsx
vendeeglobe_20201122_110000.xlsx
vendeeglobe_20201122_080000.xlsx
vendeeglobe_20201122_040000.xlsx
vendeeglobe_20201121_210000.xlsx
vendeeglobe_20201121_170000.xlsx
vendeeglobe_20201121_140000.xlsx
vendeeglobe_20201121_110000.xlsx
vendeeglobe_20201121_080000.xlsx
vendeeglobe_20201121_040000.xlsx
vendeeglobe_20201120_210000.xlsx
vendeeglobe_20201120_170000.xlsx
vendeeglobe_20201120_140000.xlsx
vendeeglobe_20201120_110000.xlsx
vendeeglobe_20201120_080000.xlsx
vendeeglobe_20201120_040000.xlsx
vendeeglobe_20201119_210000.xlsx
vendeeglobe_20201119_170000.xlsx
vendeeglobe_20201119_140000.xlsx
vendeeglobe_20201119_110000.xlsx
vendeeglobe_20201119_080000.xlsx
vendeeglobe_20201119_040000.xlsx
vendeeglobe_20201118_210000.xlsx
vendeeglobe_20201118_170000.xlsx
vendeeglobe_20201118_140000.xlsx
vendeeglobe_20201118_110000.xlsx
vendeeglobe_20201118_080000.xlsx
vendeeglobe_20201118_040000.xlsx
vendeeglob